<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

In [1]:
# IPython libraries be used to diplay and parts
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
# Camera and servo interface foe Jetbot robot car
from jetbot import Robot, Camera, bgr8_to_jpeg
# Python base package for image annotations
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time
from servoserial import ServoSerial
import threading
# Kill pthread
import inspect
import ctypes

In [2]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot():
    print(f'x_slider, {x_slider.value}')
    uuid = xy_uuid(x_slider.value, y_slider.value)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)


Directories not created becasue they already exist


In [3]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')
button_layout = widgets.Layout(width='100px', height='40px')
cap_button = widgets.Button(description='shot', button_style='success', layout=button_layout)

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)
cap_button.on_click(lambda x: save_snapshot())
display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider, cap_button)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0, step=0.001)

FloatSlider(value=0.0, description='y', max=1.0, min=-1.0, step=0.001)

Button(button_style='success', description='shot', layout=Layout(height='40px', width='100px'), style=ButtonSt…

In [7]:
camera.unobserve_all()
del camera